<a href="https://colab.research.google.com/github/Dykim991222/MLstudy/blob/main/%ED%85%8D%EC%8A%A4%ED%8A%B8%EC%A0%84%EC%B2%98%EB%A6%AC(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 텍스트 전처리
텍스트를 자연어 처리를 위해 용도에 맞도록 사전에 표준화 하는 작업.

텍스트 내 정보를 유지하고, 중복을 제거하여 분석 효율성을 높이기 위해 전처리를 수행

1. 토큰화
2. 품사 부착(Pos Tagging)
3. 개체명 인식(NER, Named Entity Recognition)
4. 원형 복원(stemming & Lemmatization)
5. 불용어 처리(Stopword)


In [5]:
import requests
from bs4 import BeautifulSoup

In [6]:
url = 'https://www.forbes.com/sites/adrianbridgwater/2019/04/15/what-drove-the-ai-renaissance/#4a0130481f25'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

In [7]:
eng_news = soup.select('p') #[class = 'speakable-paragraph']
eng_text = eng_news[3].get_text()

eng_text

'IBM CEO Ginni Rometty has already proclaimed that AI will change 100 percent of jobs over the next decade.'

1. 토큰화

텍스트를 자연어 처리하기 위해 분리하는것.

토큰화는 단어별로 분리하는 "단어 토큰화(Word Tokenization)"과 문장별로 분리하는 "문장 토큰화(Sentence Tokenization)"으로 구분


In [ ]:
!pip install nltk

In [30]:
import nltk
nltk.download('punkt') # 문장 토큰화에 사용
nltk.download('punkt_tab') # punkt가 오류날 시 보조적으로 설치
nltk.download('omw-1.4') # open multilingual wordnet, (동의어, 반의어, 유의어)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [10]:
text = "James is working at Disney in -London"

In [11]:
# word_tokenize() : 단어와 구두점(온점, 컴마, 물음표, 세미콜론, 느낌표)으로 토큰화
from nltk.tokenize import word_tokenize

word_tokens = word_tokenize(text)
print(word_tokens)

['James', 'is', 'working', 'at', 'Disney', 'in', '-London']


In [12]:
# WordPunctTokenizer() : 알파벳과 알파벳이 아닌 문자를 구분
from nltk.tokenize import WordPunctTokenizer

wordpuncttoken = WordPunctTokenizer().tokenize(text)
print(wordpuncttoken)

['James', 'is', 'working', 'at', 'Disney', 'in', '-', 'London']


In [13]:
# WordPunctTokenizer을 통해 알파벳만 뽑아보기
wordpuncttoken = [w for w in wordpuncttoken if w.isalpha()]
wordpuncttoken

['James', 'is', 'working', 'at', 'Disney', 'in', 'London']

In [14]:
# TreebankWordTokenizer() : 정규표현식에 기반한 토큰화
from nltk.tokenize import TreebankWordTokenizer

treebankwordtoken = TreebankWordTokenizer().tokenize(text)
print(treebankwordtoken)

['James', 'is', 'working', 'at', 'Disney', 'in', '-London']


2. 영문 품사 부착(PoS tagging)

각 토큰에 품사 정보를 추가

분석시에 불필요한 품사를 제거하거나(ex. 조사, 접속사) 필요한 품사를 필터링하기 위해 사용

In [ ]:
import nltk
nltk.download('all') # averaged_perceptron_tagger 설치가 /root/nltk_data를 환경인식 못하는 경우가 있음. 그때 현재 환경에 싹다 까는 것

In [16]:
from nltk.tokenize import word_tokenize
text = "James is working at Disney in London"
word_tokens = word_tokenize(text)

# 품사 태깅(PoS tagging)
from nltk import pos_tag

taggedToken = pos_tag(word_tokens)
print(taggedToken)

[('James', 'NNP'), ('is', 'VBZ'), ('working', 'VBG'), ('at', 'IN'), ('Disney', 'NNP'), ('in', 'IN'), ('London', 'NNP')]


3. 개체명 인식(NER, Named Entity Recognition)

이름을 가진 개체(named entity)를 인식하겠다는 것을 의미함.

각 토큰의 개체구분(기관, 인물, 지역, 날짜 등) 태그를 부착

텍스트가 무엇과 관련있는지 구분하기 위해 사용

ex) apple(fruit) vs apple(company)

In [17]:
nltk.download('words')
nltk.download('maxent_ne_chunker') # NER 데이터

[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!


True

In [18]:
taggedToken

[('James', 'NNP'),
 ('is', 'VBZ'),
 ('working', 'VBG'),
 ('at', 'IN'),
 ('Disney', 'NNP'),
 ('in', 'IN'),
 ('London', 'NNP')]

In [19]:
from nltk import ne_chunk
neToken = ne_chunk(taggedToken)
print(neToken)

(S
  (PERSON James/NNP)
  is/VBZ
  working/VBG
  at/IN
  (ORGANIZATION Disney/NNP)
  in/IN
  (GPE London/NNP))


4. 원형복원(Stemming & Lemmatization)

각 토큰의 원형을 복원함으로써 토큰을 표준화함 -> 불필요한 데이터의 중복을 방지

어간추출(stemming) : 품사를 무시하고 규칙에 기반하여 어간을 추출

표제어 추출(Lemmatization) : 품사정보를 유지하여 표제어 추출

- 어간추출(Stemming)

In [20]:
from nltk.stem import PorterStemmer
ps = PorterStemmer() # 어간을 추출하는 객체를 정의

In [21]:
print('running -> ' + ps.stem('running'))
print('believes -> ' + ps.stem('believes'))
print('using -> ' + ps.stem('using'))
print("conversation ->" + ps.stem('conversation'))
print('organization ->'+ ps.stem('organization'))
print('studies -> '+ ps.stem("studies"))

running -> run
believes -> believ
using -> use
conversation ->convers
organization ->organ
studies -> studi


- 표제어 추출(Lemmatization)

In [22]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [23]:
from nltk.stem import WordNetLemmatizer
wl = WordNetLemmatizer() # Wordnet 데이터 정보로 표제어 변환

In [24]:
print('running -> ' + wl.lemmatize('running'))
print('believes -> ' + wl.lemmatize('believes'))
print('using -> ' + wl.lemmatize('using'))
print("conversation ->" + wl.lemmatize('conversation'))
print('organization ->'+ wl.lemmatize('organization'))
print('studies -> '+ wl.lemmatize("studies"))

running -> running
believes -> belief
using -> using
conversation ->conversation
organization ->organization
studies -> study


5. 불용어 처리(Stopword)

In [25]:
taggedToken

[('James', 'NNP'),
 ('is', 'VBZ'),
 ('working', 'VBG'),
 ('at', 'IN'),
 ('Disney', 'NNP'),
 ('in', 'IN'),
 ('London', 'NNP')]

In [26]:
# 품사 기반으로 삭제할 목록
stopPos = ['IN','CC','UH','TO','MD','DT','VBZ','VBP']

# 최빈어 조회 : 최빈어를 조회하여 불용어 제거 대상을 선정
from collections import Counter
Counter(taggedToken).most_common()

[(('James', 'NNP'), 1),
 (('is', 'VBZ'), 1),
 (('working', 'VBG'), 1),
 (('at', 'IN'), 1),
 (('Disney', 'NNP'), 1),
 (('in', 'IN'), 1),
 (('London', 'NNP'), 1)]

In [27]:
# 워드 단위로 삭제
stopWord = [",","be","able"]

In [28]:
word = []
for tag in taggedToken: # 품사로 태깅된 문장 중에서
  if tag[1] not in stopPos: # 삭제할 품사리스트에 없으면
    if tag[0] not in stopWord: # 삭제할 단어리스트에 없으면
      word.append(tag[0]) # 워드 리스트에 저장하겠습니다.

print(word)

['James', 'working', 'Disney', 'London']


영문 텍스트 전처리 종합

In [29]:
import nltk
# 1. 토큰화
from nltk.tokenize import TreebankWordTokenizer
text = "Obama loves fried chicken of KFC"
token = TreebankWordTokenizer().tokenize(text)

# 2. 품사 부착(Pos tagging)
from nltk import pos_tag
TaggedToken = pos_tag(token)

# 3. 개체명 인식(NER)
# 한 문장이기에 일단 생략. 이후 실습에서 연습할 예정

# 4. 원형복원(Stemming & Lemmatization)
from nltk.stem import PorterStemmer # stem, 어간추출
from nltk.stem import WordNetLemmatizer # lemmatizer, 표제어추출
ps = PorterStemmer()
wl = WordNetLemmatizer()

StemmingList = []
for w in TaggedToken:
  StemmingList.append(ps.stem(w[0]))
print(StemmingList)

LemmatizerList = []
for w in TaggedToken:
  LemmatizerList.append(wl.lemmatize(w[0]))
print(LemmatizerList)

# 5. 불용어 처리
stopPos = ['IN']
stopWord = ['fried']

final_word = []

for tag in TaggedToken:
  if tag[1] not in stopPos:
    if tag[0] not in stopWord:
      final_word.append(tag[0])

print(final_word)

['obama', 'love', 'fri', 'chicken', 'of', 'kfc']
['Obama', 'love', 'fried', 'chicken', 'of', 'KFC']
['Obama', 'loves', 'chicken', 'KFC']
